# Imports

In [ ]:
import numpy as np
import pandas as pd

import torch

from captum.robust import PGD

import random

In [ ]:
import sys

sys.path.append('/mnt/home/rheinrich/taaowpf')

from data.lstm.wpf_dataset_single_turbine_gefcom import WPF_SingleTurbine_DataModule
from models.lstm.lstm import WPF_AutoencoderLSTM
from robustness_evaluation.robustness_scores import MSELossBatch

## Set Hyperparameters for Model & Training

In [ ]:
config = {
    'checkpoint_path_normal_training': '/mnt/home/rheinrich/taaowpf/models/lstm/checkpoints_normal_training/best_lstm_model_gefcom_normal_training_zone1.ckpt',
    'checkpoint_path_adversarial_training': '/mnt/home/rheinrich/taaowpf/models/lstm/checkpoints_adversarial_training/best_lstm_model_gefcom_adversarial_training_zone1.ckpt',
    'forecast_horizon': 8, # 8 hour ahead wind power forecast 
    'n_past_timesteps': 12, # including current time step
    'hidden_size': 32,
    'num_layers': 1,
    'batch_size': 256,
    'num_workers': 32,
    'learning_rate': 0.01,
    'p_adv_training': 0.0,
    'eps_adv_training': 0.15, 
    'step_num_adv_training': 100,
    'norm_adv_training': 'Linf',
    'target_attacker': [0.25, 0.40, 0.50, 0.60, 0.65, 0.72, 0.78, 0.82], # increasing
    'step_num_noise_attack': 100,
    'eps_pgd_attack_list': [0.15, 1.0, 2.0, 3.0],
}

# Initialize DataModule

In [ ]:
data_dir = '/mnt/home/rheinrich/taaowpf/data/lstm/Gefcom2014_Wind/gefcom2014_W_100m_zone1.csv'

In [ ]:
datamodule = WPF_SingleTurbine_DataModule(data_dir = data_dir,
                                          forecast_horizon = config['forecast_horizon'],  
                                          n_past_timesteps = config['n_past_timesteps'],
                                          batch_size = config['batch_size'], 
                                          num_workers = config['num_workers'],
                                         )

In [ ]:
datamodule.setup()

## Load examplary sample

In [ ]:
input_windspeed, input_windpower, target = datamodule.test_dataset.__getitem__(1950)

In [ ]:
input_windspeed, input_windpower = input_windspeed.unsqueeze(0), input_windpower.unsqueeze(0)

#### Destandardized wind speed of the original input sample

In [ ]:
input_windspeed_destandardized = (input_windspeed * datamodule.std_windspeed) + datamodule.mean_windspeed

In [ ]:
input_windspeed_destandardized = input_windspeed_destandardized.squeeze().detach().numpy()

# Initialize Models

## Load best model checkpoint

#### Normal Training

In [ ]:
model_normal_training = WPF_AutoencoderLSTM.load_from_checkpoint(config['checkpoint_path_normal_training'],
                                                                forecast_horizon = config['forecast_horizon'],
                                                                n_past_timesteps = config['n_past_timesteps'],
                                                                hidden_size = config['hidden_size'],
                                                                num_layers = config['num_layers'],
                                                                learning_rate= config['learning_rate'],
                                                                p_adv_training = config['p_adv_training'],
                                                                eps_adv_training = config['eps_adv_training'],
                                                                step_num_adv_training = config['step_num_adv_training'],
                                                                norm_adv_training = config['norm_adv_training']
                                                              )

### Set models to evaluation mode

In [ ]:
model_normal_training.eval()

## Model prediction for original input

#### Normal Training

In [ ]:
with torch.no_grad():
    prediction_normal_training = model_normal_training(input_windspeed, input_windpower)

# Adversarial Robustness Evaluation: Example Attack

In [ ]:
# set seeds for reproducibility
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

#### Set lower bound for perturbations, such that perturbed wind speed is never negative

In [ ]:
lower_bound = (0 - datamodule.mean_windspeed) / datamodule.std_windspeed

#### Target of the attacker

In [ ]:
target_attacker = torch.tensor(config['target_attacker'])

#### DataFrame containing the ground truth target, the attacker's target such as the prediction of the model for the original input

##### Normal Training

In [ ]:
results_normal_training = pd.DataFrame(target.detach().numpy(), columns = ['ground truth'])
results_normal_training["attacker's target"] = target_attacker.numpy()
results_normal_training['original prediction'] = prediction_normal_training.detach().numpy()[0]

## Adversarial Robustness: Targeted PGD Attack (example attack)

In [ ]:
for eps_pgd_attack in config['eps_pgd_attack_list']:
    #### Initialize PGD Attack
    pgd_normal_training = PGD(model_normal_training, MSELossBatch(), lower_bound = lower_bound)
    
    #### Generate perturbed input
    input_windspeed_pgd_normal_training = pgd_normal_training.perturb(inputs = input_windspeed,
                                                                  radius = eps_pgd_attack,
                                                                  step_num = config['step_num_adv_training'],
                                                                  step_size = 2 * eps_pgd_attack / config['step_num_adv_training'],
                                                                  target = target_attacker.unsqueeze(0), 
                                                                  targeted = True,
                                                                  norm = config['norm_adv_training'],
                                                                  additional_forward_args = input_windpower
                                                                 )
    #### Model prediction for perturbed input
    with torch.no_grad():
        prediction_pgd_normal_training = model_normal_training(input_windspeed_pgd_normal_training, input_windpower)
        
    #### DataFrame with results (ground truth target, attacker's target, prediction for original input, prediction for perturbed input)
    results_pgd_normal_training = results_normal_training.copy()
    results_pgd_normal_training['attacked prediction'] = prediction_pgd_normal_training.detach().numpy()[0]
    
    #### Destandardized wind speed of the perturbed input sample
    input_windspeed_pgd_normal_training_destandardized = (input_windspeed_pgd_normal_training * datamodule.std_windspeed) + datamodule.mean_windspeed
    
    input_windspeed_pgd_normal_training_destandardized = input_windspeed_pgd_normal_training_destandardized.squeeze().detach().numpy()
    
    inputs_pgd_normal_training = pd.DataFrame([input_windspeed_destandardized, input_windspeed_pgd_normal_training_destandardized]).T
    inputs_pgd_normal_training.columns = ['original wind speed', 'perturbed wind speed']
    
    # Save all results as CSV file
    
    ### Predictions
    results_pgd_normal_training.to_csv(f"single-turbine_gefcom_example-attack_predictions_pgd_normal-training_eps{str(eps_pgd_attack)}.csv", index = False)
    
    ### Inputs
    inputs_pgd_normal_training.to_csv(f"single-turbine_gefcom_example-attack_inputs_pgd_normal-training_eps{str(eps_pgd_attack)}.csv", index = False)